In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Loading functions
import os
from monai.networks.nets import SwinUNETR
from functools import partial
from monai.data import DataLoader, decollate_batch
from src.get_data import CustomDataset
from monai.transforms import (
    AsDiscrete,
    Activations,
    MapTransform,
)
from scipy import ndimage

import torch
import torch.nn.parallel
import logging
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np

from monai.inferers import sliding_window_inference
from monai import transforms
from monai.transforms import (
    AsDiscrete,
    Activations,
)
import torch
import wandb

from src.get_data import CustomDataset
from monai.data import DataLoader

from src.custom_transforms import (
    ConvertToMultiChannelBasedOnAnotatedInfiltration,
    masked,
)


In [3]:
######################
# Crear el modelo
######################

### Hyperparameter
roi = (128, 128, 128)  # (128, 128, 128)

# Create Swin transformer
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = SwinUNETR(
    img_size=roi,
    in_channels=11,
    out_channels=2,  # mdificar con edema
    feature_size=48,
    drop_rate=0.0,
    attn_drop_rate=0.0,
    dropout_path_rate=0.0,
    use_checkpoint=True,
)#.to(device)

/home/training2/miniconda3/envs/monai_env/lib/python3.11/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
  warn_deprecated(argname, msg, warning_category)


In [4]:
# run = wandb.init()
# artifact = run.use_artifact('mlops-team89/Swin_UPENN_10cases/q7fzlnx3_best_model:v0', type='model')
# artifact_dir = artifact.download()
# print(artifact_dir)

In [5]:
#model.load_state_dict(torch.load(os.path.join(artifact_dir, "model.pt"), map_location=torch.device('cpu')))

# model_path = "Dataset/model.pt"
model_path = "artifacts/q7fzlnx3_best_model:v0/model.pt" #'Dataset/model_dataset_330_30_96x96x96_48f_v02.pt' # 5mm - mjkearkn_best_model-v0 / 10mm - ip0bojmx_best_model-v0

# Load the model on CPU
loaded_model = torch.load(model_path, map_location=torch.device(device))["state_dict"]
# model.load_state_dict(torch.load(model_path)["state_dict"])

# Load the state dictionary into the model
model.load_state_dict(loaded_model)

# # Move the model to the desired device (e.g., GPU) if available
# # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# # Set the model to evaluation mode
# model.eval()

/tmp/ipykernel_1369362/2056417573.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(model_path, map_location=torch.device(device))["state_dict"]


<All keys matched successfully>

### Datset

In [6]:
 #Transformaciones 240, 240, 155
roi = (128, 128, 128) 
val_transform = transforms.Compose(
    [
        transforms.LoadImaged(keys=["image", "label"]), #Leer imagenes 
        ConvertToMultiChannelBasedOnAnotatedInfiltration(keys="label"),
        # transforms.CropForegroundd(
        #     keys=["image", "label"],
        #     source_key="label",
        #     k_divisible=[roi[0], roi[1], roi[2]],
        # ),
        transforms.RandSpatialCropd(
            keys=["image", "label"],
            roi_size=[-1, -1, -1],
            random_size=False,
        ),   
        transforms.NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True), #Normalizar intensidades
    ]
)

# Dado un arreglo de paths de imagenes devuelve un tensor torch.Size([canales, x, y, z])
def get_image_data(image, transform):
    data = transforms.apply_transform(
                transform,
                data= {"image":image},
            )
    return data["image"]  

In [7]:
dataset_path = "./Dataset/Dataset_10_1_casos"
val_set = CustomDataset(
        dataset_path, section="train_all", transform=val_transform
    )  # v_transform
val_loader = DataLoader(val_set, batch_size=1, shuffle=False, num_workers=0)

im_v=val_set[0]
print(im_v['image'].shape)
print(im_v['label'].shape)

Found 11 images and 11 labels.
torch.Size([11, 240, 240, 155])
torch.Size([2, 240, 240, 155])


In [8]:
# from temperature import ModelWithTemperature

# scaled_model = ModelWithTemperature(model)
# scaled_model.set_temperature(val_loader)

In [9]:
from temperature2 import find_optimal_temperature
from monai.losses import DiceLoss


criterion = torch.nn.BCEWithLogitsLoss() #BCEWithLogitsLoss()
# criterion = DiceLoss(to_onehot_y=False, sigmoid=True)

optimal_temperature = find_optimal_temperature(model, val_loader, criterion)

print(f'Optimal temperature: {optimal_temperature}')

100%|██████████| 11/11 [02:22<00:00, 12.97s/it]


Iteración: 1, Temperatura: 5.599999904632568, Loss: 0.009853550232946873


100%|██████████| 11/11 [02:35<00:00, 14.14s/it]


Iteración: 2, Temperatura: 5.6000261306762695, Loss: 0.009853536263108253


100%|██████████| 11/11 [03:04<00:00, 16.77s/it]


...

Iteración: 98, Temperatura: 5.602543830871582, Loss: 0.009852212853729725


100%|██████████| 11/11 [03:05<00:00, 16.85s/it]


Iteración: 99, Temperatura: 5.602570056915283, Loss: 0.00985219981521368


100%|██████████| 11/11 [03:05<00:00, 16.87s/it]


Iteración: 100, Temperatura: 5.602596282958984, Loss: 0.009852185845375061
Optimal temperature: 5.6026225090026855
